# Lab 6A: CNN Architectures and Transfer Learning

The learning objectives for this lab exercise are as follows:
1. Customize the standard CNN Network to a targeted task
2. Perform different kinds of transfer learning:
    1. Train from scratch
    2. Finetune the whole model
    3. Finetune the upper layers of the model
    4. As a feature extractor

In practice, it is common to use a **standard CNN architectures** such that ResNet, MNASNet, ResNeXt, EfficientNet, etc. to build a model. The effectiveness of these network architectures has been well attested for a wide range of applications. 

Rather than training from scratch, it is advisable to use **transfer learning** by training on top of a standard model that has been **pretrained** on the ImageNet dataset. Transfer learning reduces overfitting and improves the generalization performance of the trained model, especially when the training set for the targeted task is small. The [`torchvision.models`](https://pytorch.org/vision/stable/models.html) package contains these different network models that have been pre-trained on ImageNet. 

We perform transfer learning in two ways:

1. *Finetuning the convnet*: Instead of random initialization,  initialize the network with the pretrained network. 

2. *Fixed feature extractor*: Freeze the weights for all of the layers of the network except for the final fully connected (fc) layer. Replace the last fc layer so that the output size is the same as the number of classes for the new task. The new layer is initialized with random weights and only this layer is trained.

Mount google drive onto virtual machine

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Change current directory to Lab 6

In [ ]:
cd "/content/gdrive/My Drive/UCCD3074_Labs/UCCD3074_Lab6"

Load required libraries

In [ ]:
import numpy as np
import torchvision.models as models

import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchsummary import summary

from cifar10 import CIFAR10

---
## Helper Functions

Define the train function

In [ ]:
loss_iter = 1

def train(net, num_epochs, lr=0.1, momentum=0.9, verbose=True):
    
    history = []
    
    loss_iterations = int(np.ceil(len(trainloader)/loss_iter))
    
    # transfer model to GPU
    if torch.cuda.is_available():
        net = net.cuda()
    
    # set the optimizer
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)
    
    # set to training mode
    net.train()

    # train the network
    for e in range(num_epochs):    

        running_loss = 0.0
        running_count = 0.0

        for i, (inputs, labels) in enumerate(trainloader):

            # Clear all the gradient to 0
            optimizer.zero_grad()

            # transfer data to GPU
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            # forward propagation to get h
            outs = net(inputs)

            # compute loss 
            loss = F.cross_entropy(outs, labels)

            # backpropagation to get dw
            loss.backward()

            # update w
            optimizer.step()

            # get the loss
            running_loss += loss.item()
            running_count += 1

             # display the averaged loss value 
            if i % loss_iterations == loss_iterations-1 or i == len(trainloader) - 1:                
                train_loss = running_loss / running_count
                running_loss = 0. 
                running_count = 0.
                if verbose:
                    print(f'[Epoch {e+1:2d}/{num_epochs:d} Iter {i+1:5d}/{len(trainloader)}]: train_loss = {train_loss:.4f}')       
                
                history.append(train_loss)
    
    return history

Define the evaluate function

In [ ]:
def evaluate(net):
    # set to evaluation mode
    net.eval()
    
    # running_correct
    running_corrects = 0
    
    for inputs, targets in testloader:
        
        # transfer to the GPU
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            targets = targets.cuda()
        
        # perform prediction (no need to compute gradient)
        with torch.no_grad():
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            running_corrects += (targets == predicted).double().sum()
            
    print('Accuracy = {:.2f}%'.format(100*running_corrects/len(testloader.dataset)))

## 1. Load CIFAR10 dataset

Here, we use a sub-sample of CIFAR10 where we use a sub-sample of 1000 training and testing samples. The sample size is small and hence is expected to face overfitting issue. Using a pretrained model alleviates the problem.

In [ ]:
# transform the model
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# dataset
trainset = CIFAR10(train=True, download=True, transform=transform, num_samples=1000)
testset  = CIFAR10(train=False, download=True, transform=transform, num_samples=1000)

# dataloader]
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
testloader  = DataLoader(testset, batch_size=128, shuffle=True, num_workers=2)

## 2. The ResNet50 model

In this section, we shall build our network using a standard network architectures. We customize a pre-trained [ResNet50](https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet50.html#resnet50) by replacing its classifier layer, i.e., the last fully connected layer with our own. The original ImageNet classifier is designed to classify 1000 output classes whereas our CIFAR10 classifier handles only 10 classes. 

### Using the pre-trained models

First, let's learn how to load and use a pre-trained model as it is. The following table lists the pretrained models for ResNet50 together their reported accurcies on ImageNet-1K with single crops.

|**weight**|**Acc@1**|**Acc@5**|**Params**|
|:---:|:---:|:---:|:---:|
|ResNet50_Weights.IMAGENET1K_V1|76.13|92.862|25.6MB|
|ResNet50_Weights.IMAGENET1K_V2|80.858|95.434|25.6MB|

where IMAGENET1K_V2 improves upon IMAGENET1K_V1 by using a new [training recipe](https://pytorch.org/blog/how-to-train-state-of-the-art-models-using-torchvision-latest-primitives/)


To specify the pretrained model, you can use the predefined constant:

In [ ]:
# ... import required module ...

In [ ]:
# ... load pretrained resnet18 model with predefined constant argument ...

or a string argument.

In [ ]:
# ... load pretrained resnet18 model with string argument ...

### Inferencing with the pretrained model

Some pretrained model needs specific preprocessing steps (e.g., resize into a specific resolution / rescale the values, etc.). The preprocessing steps vary depending on how the model was trained. The necessary information for inference transforms are provided on the weight documentation. But to simplify inference, `TorchVision` also bundle a `transform` utility into `ResNet.Weights`.

In [ ]:
# ... load the preprocess function ...

In [ ]:
# ... read image ...

# ... preprocess image ...

# ... create the batch dimension ...

print('Shape of x after unsqueezing:', x.shape)

Perform inference with the pretrained model. The classes of the pretrained model can be found at `weights.meta['categories']`.

In [ ]:
# ... perform inference

# ... get predicted class ...

# ... print predicted class label ...


### Customizing ResNet50

In the following, we shall replace the last layer with a new classifier layer. The pre-trained model is designed to classify ImageNet's 1000 image categories. In the following, we shall customize it to classify Cifar10's 10 classes. First, let's look at how ResNet50 is implemented in PyTorch.

In [ ]:
# ... print network ...

In [ ]:
# ... print the children of network ...

Here are some observations:
* `conv1`, `bn1`, `relu` and `maxpool` are the *stem* network
* There are 4 *blocks* in the network, namely `layer1`, `layer2`, `layer3` and `layer4`.
* Each of the block contains two convolutional layers. 
* The second last layer (`avgpool`) performs *global average pooling* to average out the spatial dimensions. 
* The last layer (`fc`) is a linear layer that functions as a classifier. **This is the layer that we want to replace to fit our model**.

To customize the network, we need to replace the `fc` layer with our own classifier layer.


In [ ]:
def build_network(weights=None):
    # ... customize the resnet18 network for CIFAR 10
    return net

Let's visualize what we have built. Note that the last layer of the network (`fc`) now has 10 instead of 1000 neurons.

In [ ]:
print(build_network())

---
### Model 1: Training from scratch

Let's build the network **without** loading the pretrained model. To do this, we set `weights=None`.

In [ ]:
# ... build network without pretraining ...

Train the model and save the training loss history into `history1`.

In [ ]:
history1 = train(net1, num_epochs=30, lr=0.01, momentum=0.8)

Evaluate the model

In [ ]:
evaluate(net1)

---
### Model 2: Finetuning the pretrained model

Typically, a standard network come with a pretrained model trained on ImageNet's large-scale dataset for the image classification task. 
* In the following, we shall load resnet50 with the pretrained model and use it to **initialize** the network.  To do this, we set `pretrained=True`.
* The training will update the parameters **all layers** of the network.

For Windows system, the pretrained model will be saved to the following directory: `C:\Users\<user name>\.cache\torch\checkpoints`. A PyTorch model has an extension of `.pt` or `.pth`. 

In [ ]:
# ... build network with IMAGENET1K_V2 pretrained model ...

By default, all the layers are set to `requires_grad=True`

In [ ]:
# ... display the requires_grad for all layers ...

Train the model and save into `history2`.

In [ ]:
history2 = train(net2, num_epochs=30, lr=0.01, momentum=0.8)

Evaluate the network

In [ ]:
evaluate(net2)

---
### Model 3: As a fixed feature extractor

When the dataset is too small, fine-tuning the model may still incur overfitting. In this case, you may want to try to use the pretrained as a fixed feature extractor where we train only the  classifier layer (i.e., **last layer**) that we have newly inserted into the network.

In [ ]:
# ... build network with IMAGENET1K_V2 pretrained model ...

We set `requires_grad=False` for all parameters except for the newly replaced layer `fc`, i.e., the last two parameters in `resnet.parameters()`.

In [ ]:
# ... set requires grad to FALSE for all parameters except for the newly replace layer ...

In [ ]:
# ... confirm that the gradients have been set correctly ...

Train the model and save into `history3`.

In [ ]:
history3 = train(net3, num_epochs=30, lr=0.01, momentum=0.8)

Evaluate the model

In [ ]:
evaluate(net3)

---
### Model 4: Finetuning the top few layers

We can also tune the top few layers of the network. The following tunes all the layers in the block `layer 4` as well as the `fc` layer.


In [ ]:
# ... build network with IMAGENET1K_V2 pretrained model ...

Then, we freeze all tha layers except for `layer4` and `fc` layers

In [ ]:
# ... set requires grad to FALSE for all parameters except for layer4 and fc layers ...

In [ ]:
# ... confirm that the gradients have been set correctly ...

Train the model and save into `history4`.

In [ ]:
history4 = train(net4, num_epochs=30, lr=0.01, momentum=0.8)

Evaluate the model

In [ ]:
evaluate(net4)

### Plotting training loss

Lastly, we plot the training loss history for each of the training schemes above.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history1, label='From scratch')
plt.plot(history2, label='Finetuning the pretrained model')
plt.plot(history3, label='As a fixed extractor')
plt.plot(history4, label='Finetuning the top few layers')
plt.legend()
plt.show()

## Exercise

You can try with different network architectures (e.g., [EfficientNet-B0](https://pytorch.org/vision/stable/models/efficientnet.html)) and see if it results in higher test accuracy.

> *The list of all pre-trained models in PyTorch is listed in this [Table](https://pytorch.org/vision/stable/models.html#table-of-all-available-classification-weights).*


In [ ]:
def build_network(weights='IMAGENET1K_V1'):
    # ... your answer here ...
    return net

In [ ]:
efficientNet = build_network() 

In [ ]:
history5 = train(efficientNet, num_epochs=30, lr=0.01, momentum=0.8)

In [ ]:
evaluate(efficientNet)